##Social Recommendation using Graph

In [20]:
from py2neo import Node, Relationship, Graph, schema
from pymongo import MongoClient

In [3]:
client = MongoClient('localhost', 27017)
db = client.yelp_dataset
business = db.business

usBoundingBox = ((-124.848974, 24.396308), (-66.885444, 49.384358))
businessCursor = business.find({'categories':'Restaurants', 
                                'longitude': {'$lte': -66.885444}, 
                                'longitude': {'$gte': -124.848974}})

allBusiness = [b for b in businessCursor]

reviews = db.reviews
businessIds = [b['business_id'] for b in allBusiness]
reviewCur = reviews.find({'business_id':{'$in':businessIds}})
usRestaurantReviews = [r for r in reviewCur]

In [28]:
graph = Graph()
graph.cypher.execute("CREATE INDEX ON :Users(name)")
graph.cypher.execute("CREATE INDEX ON :Businesses(name)")

In [29]:
graph = Graph()
for bId in businessIds:
    bsness = Node("Businesses", name=bId)
    graph.create(bsness)
    


In [30]:
uniqueUsers = list(set([rev['user_id'] for rev in usRestaurantReviews]))
for user in uniqueUsers:
    userNode = Node("Users", name=user)
    graph.create(userNode)


In [41]:
count = 0
for rev in usRestaurantReviews:
    star = rev['stars']
    if star >= 4:
        user = graph.merge_one('Users', 'name', rev['user_id'])
        business = graph.merge_one('Businesses', 'name', rev['business_id'])
        likes = Relationship(user, 'likes', business)
        graph.create(likes)
        count += 1
        
print count


652216
